# Observational Study VOI Validation Notebook

This notebook validates the Observational Study VOI implementation in voiage by demonstrating its functionality with practical examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from voiage.methods.observational import voi_observational
from voiage.schema import ParameterSet

## Define an Observational Study Modeler

First, let's create a function that models the impact of observational data.

In [ ]:
def simulate_observational_study(psa_samples, study_design, bias_models):
    """Simulate an observational study and evaluate economic outcomes.
    
    This is a simplified example. In practice, this would involve:
    - Defining the observational study design
    - Modeling potential biases and their impact on parameter estimation
    - Simulating the observational data collection process
    - Specifying how this data updates decision model parameters
    """
    # Extract parameters
    if isinstance(psa_samples, ParameterSet):
        params = psa_samples.parameters
    else:
        params = psa_samples
    
    # Get treatment effectiveness
    if 'effectiveness' in params:
        effectiveness = params['effectiveness']
    else:
        effectiveness = np.full_like(next(iter(params.values())), 0.7)
        
    # Get treatment costs
    if 'cost' in params:
        cost = params['cost']
    else:
        cost = np.full_like(next(iter(params.values())), 5000)
    
    # Willingness-to-pay threshold (£ per QALY)
    wtp_threshold = 20000
    
    # Calculate net benefits for each strategy
    n_samples = len(effectiveness)
    n_strategies = 2
    
    net_benefits = np.zeros((n_samples, n_strategies))
    
    # Standard care net benefit
    net_benefits[:, 0] = effectiveness * 0.8 * wtp_threshold - cost * 0.8
    # New treatment net benefit (slightly better)
    net_benefits[:, 1] = effectiveness * 1.0 * wtp_threshold - cost * 1.0
    
    # Create ValueArray
    import xarray as xr
    dataset = xr.Dataset(
        {"net_benefit": (("n_samples", "n_strategies"), net_benefits)},
        coords={
            "n_samples": np.arange(n_samples),
            "n_strategies": np.arange(n_strategies),
            "strategy": ("n_strategies", ["Standard Care", "New Treatment"])
        }
    )
    
    from voiage.schema import ValueArray
    return ValueArray(dataset=dataset)

## Generate Parameter Samples

Let's generate parameter samples for our probabilistic sensitivity analysis (PSA).

In [ ]:
# Generate parameter samples
np.random.seed(42)  # For reproducibility
n_samples = 1000

# Treatment effectiveness (QALYs)
effectiveness = np.random.normal(0.7, 0.1, n_samples)

# Treatment costs (£)
cost = np.random.normal(5000, 500, n_samples)

# Create ParameterSet
parameters = {
    'effectiveness': effectiveness,
    'cost': cost
}

parameter_set = ParameterSet.from_numpy_or_dict(parameters)
print(f"Generated {parameter_set.n_samples} parameter samples")

## Define Observational Study Design

Let's define the observational study design.

In [ ]:
# Define observational study design
observational_study_design = {
    "study_type": "cohort",
    "sample_size": 5000,
    "variables_collected": ["treatment", "outcome", "age", "gender", "comorbidities"],
    "follow_up_duration": 5,  # years
    "data_source": "electronic_health_records"
}

print("Observational study design:")
for key, value in observational_study_design.items():
    print(f"  {key}: {value}")

## Define Bias Models

Let's define the bias models for our observational study.

In [ ]:
# Define bias models
bias_models = {
    "confounding": {
        "strength": 0.3,
        "variables": ["age", "comorbidities"]
    },
    "selection_bias": {
        "probability": 0.15,
        "mechanism": "differential_treatment_assignment"
    },
    "measurement_error": {
        "outcome": 0.05,
        "treatment": 0.02
    }
}

print("Bias models:")
for bias_type, bias_details in bias_models.items():
    print(f"  {bias_type}: {bias_details}")

## Calculate Observational Study VOI

Now let's calculate the Value of Information for our observational study.

In [ ]:
# Calculate observational study VOI
observational_voi_value = voi_observational(
    obs_study_modeler=simulate_observational_study,
    psa_prior=parameter_set,
    observational_study_design=observational_study_design,
    bias_models=bias_models,
    n_outer_loops=10
)

print(f"Observational Study VOI: £{observational_voi_value:,.0f}")

## Population-Adjusted Observational Study VOI

Let's calculate the population-adjusted Observational Study VOI for a larger population.

In [ ]:
# Calculate population-adjusted observational study VOI
observational_voi_pop = voi_observational(
    obs_study_modeler=simulate_observational_study,
    psa_prior=parameter_set,
    observational_study_design=observational_study_design,
    bias_models=bias_models,
    population=100000,  # 100,000 patients
    time_horizon=10,    # 10 years
    discount_rate=0.03, # 3% discount rate
    n_outer_loops=10
)

print(f"Population-adjusted Observational Study VOI: £{observational_voi_pop:,.0f}")

## Sensitivity Analysis

Let's perform a sensitivity analysis by varying the sample size of the observational study.

In [ ]:
# Vary sample size and calculate observational study VOI
sample_sizes = [1000, 2000, 5000, 10000]
voi_values = []

for sample_size in sample_sizes:
    study_design = {
        "study_type": "cohort",
        "sample_size": sample_size,
        "variables_collected": ["treatment", "outcome", "age", "gender", "comorbidities"],
        "follow_up_duration": 5,
        "data_source": "electronic_health_records"
    }
    
    voi_val = voi_observational(
        obs_study_modeler=simulate_observational_study,
        psa_prior=parameter_set,
        observational_study_design=study_design,
        bias_models=bias_models,
        n_outer_loops=5
    )
    
    voi_values.append(voi_val)
    print(f"Sample size: {sample_size:,}, Observational Study VOI: £{voi_val:,.0f}")

## Visualization

Let's visualize how Observational Study VOI changes with sample size.

In [ ]:
# Plot Observational Study VOI vs Sample Size
plt.figure(figsize=(10, 6))
plt.plot(sample_sizes, voi_values, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Sample Size')
plt.ylabel('Observational Study VOI (£)')
plt.title('Observational Study VOI vs Sample Size')
plt.grid(True, alpha=0.3)
plt.show()

## Comparison with Different Bias Scenarios

Let's compare different bias scenarios.

In [ ]:
# Compare different bias scenarios
bias_scenarios = [
    {"name": "Low Bias", "confounding": {"strength": 0.1}, "selection_bias": {"probability": 0.05}},
    {"name": "Medium Bias", "confounding": {"strength": 0.3}, "selection_bias": {"probability": 0.15}},
    {"name": "High Bias", "confounding": {"strength": 0.5}, "selection_bias": {"probability": 0.25}}
]
voi_bias_values = []

for scenario in bias_scenarios:
    bias_models = {
        "confounding": scenario["confounding"],
        "selection_bias": scenario["selection_bias"],
        "measurement_error": {"outcome": 0.05, "treatment": 0.02}
    }
    
    voi_val = voi_observational(
        obs_study_modeler=simulate_observational_study,
        psa_prior=parameter_set,
        observational_study_design=observational_study_design,
        bias_models=bias_models,
        n_outer_loops=5
    )
    
    voi_bias_values.append(voi_val)
    print(f"Scenario: {scenario['name']}, Observational Study VOI: £{voi_val:,.0f}")

## Summary

This notebook has demonstrated:

1. How to set up an observational study VOI analysis with voiage
2. How to define observational study modelers
3. How to calculate VOI for observational studies
4. How to perform population-adjusted calculations
5. How to conduct sensitivity analysis on sample size
6. How to compare different bias scenarios

The voiage observational study VOI implementation provides a flexible framework for evaluating the value of observational data collection in health economics and other domains.